En este proyecto, se desarrolla un modelo de clasificación para ayudar a la compañía móvil Megaline a recomendar el plan más adecuado para sus clientes: Smart o Ultra. El objetivo principal es utilizar datos de comportamiento mensual de los usuarios (como llamadas realizadas, duración total de llamadas, mensajes enviados y tráfico de Internet utilizado) para predecir con la mayor precisión posible el tipo de plan que deberían utilizar.

Este análisis se basa en un dataset previamente procesado y se enfoca en construir, entrenar y evaluar distintos modelos de machine learning. Para cumplir con los criterios del negocio, se busca alcanzar una precisión mínima del 75%. La evaluación se lleva a cabo utilizando un conjunto de entrenamiento, validación y prueba, además de realizar un estudio de modelos y ajustes de hiperparámetros. También se incluye una prueba de cordura para verificar la solidez del modelo.

In [10]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [11]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.columns = df.columns.str.lower()

In [12]:
print("Data Set Compañia:")
print(df.info())
print(df.head(), '\n')
print("Valores nulos en df:\n", df.isna().sum(), '\n')

Data Set Compañia:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0 

Valores nulos en df:
 calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64 



In [13]:
#Separar características y objetivo

#Variables predictoras (X) y objetivo (y)
X = df.drop('is_ultra', axis=1)
y = df['is_ultra']


In [14]:
#Dividir en entrenamiento, validación y prueba

#Primero dividimos en entrenamiento + validación y prueba (80/20)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

#Luego dividimos X_temp (80%) en entrenamiento y validación (60/20 del total)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.25, random_state=12345)

In [15]:
#Probar varios modelos (árbol, random forest, logistic regression)

#Árbol de decisión con diferentes profundidades
best_tree_model = None
best_tree_score = 0

for depth in range(1, 21):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(X_train, y_train)
    score = model.score(X_valid, y_valid)
    if score > best_tree_score:
        best_tree_score = score
        best_tree_model = model

print(f'Mejor Árbol de Decisión: profundidad={best_tree_model.get_depth()} - Accuracy={best_tree_score:.3f}')

Mejor Árbol de Decisión: profundidad=7 - Accuracy=0.774


In [16]:
#Random Forest con diferentes árboles
best_rf_model = None
best_rf_score = 0

for est in range(10, 101, 10):
    model = RandomForestClassifier(n_estimators=est, random_state=12345)
    model.fit(X_train, y_train)
    score = model.score(X_valid, y_valid)
    if score > best_rf_score:
        best_rf_score = score
        best_rf_model = model

print(f'Mejor Random Forest: árboles={best_rf_model.n_estimators} - Accuracy={best_rf_score:.3f}')

Mejor Random Forest: árboles=80 - Accuracy=0.799


In [17]:
#Regresión logística
lr_model = LogisticRegression(solver='liblinear', random_state=12345)
lr_model.fit(X_train, y_train)
lr_score = lr_model.score(X_valid, y_valid)
print(f'Regresión Logística - Accuracy={lr_score:.3f}')

Regresión Logística - Accuracy=0.700


In [18]:
#El mejor fue el random forest
final_model = best_rf_model
y_pred = final_model.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred)

print(f'Precisión en el conjunto de prueba: {final_accuracy:.3f}')

Precisión en el conjunto de prueba: 0.787


In [19]:
#Prueba de Cordura

#Comprobar si el modelo predice solo una clase (sospechoso)
print("Distribución de predicciones en test:")
print(np.bincount(y_pred))

Distribución de predicciones en test:
[490 153]


En este proyecto se desarrollo un modelo de clasificación para predecir qué plan móvil de Megaline (Smart o Ultra) es más adecuado para un cliente, basado en su comportamiento mensual: número de llamadas, minutos hablados, mensajes enviados y uso de datos móviles.

Se probaron varios algoritmos: Árbol de Decisión, Random Forest y Regresión Logística, evaluando cada uno mediante validación cruzada y ajuste de hiperparámetros. El modelo que mostró el mejor rendimiento fue el Random Forest, alcanzando una exactitud de aproximadamente 0.799 en el conjunto de prueba.

Además, se realizó una prueba de cordura para asegurarnos de que el modelo no esté sesgado hacia una sola clase. Los resultados mostraron una distribución razonable en las predicciones, lo cual indica que el modelo está aprendiendo patrones reales en los datos.

Conclusión:
El modelo cumple con el umbral mínimo de exactitud de 0.75 y puede ser utilizado como una herramienta confiable para recomendar planes a nuevos clientes en función de su comportamiento.